# **OpenCV**

In [ ]:
import cv2

# #Load the image
# from google.colab import drive
# drive.mount('/content/drive')

# read the image
image = cv2.imread('/content/drive/MyDrive/img_cv/faces_2.jpg')
image.shape
# covert image into gray image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# face detector
face_detector = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_frontalface_default.xml')
eye_detector = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_eye.xml')
# check detector is
if face_detector.empty():
  raise IOError('Unable to load the face cascade classifier xml file')

if eye_detector.empty():
  raise IOError('Unable to load the eye cascade classifier xml file')

# detection face:
detections = face_detector.detectMultiScale(gray_image, scaleFactor= 1.06, minNeighbors=3, minSize=(15, 15))
eye_detections = eye_detector.detectMultiScale(gray_image, scaleFactor= 1.3, minNeighbors=4, minSize=(3, 3))

# draw the dectecte array on image
# x, y, w, h are cordinate 4 dimen of dectected array
# image: the input image we draw in
# (x,y) top-left corner of detected array
# (x+w,y+h) bottom-right corner of detected array
# (0, 255, 0), 2: green RGB, thickness = 2
for (x, y, w, h) in detections:
  cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
for (x, y, w, h) in eye_detections:
  cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)
cv2_imshow(image)

# **DLib - HOG**

In [ ]:
import dlib
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
image = cv2.imread('/content/drive/MyDrive/img_cv/test_img/img/faces.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_detector_hog = dlib.get_frontal_face_detector()

# this hog detection not required the input img is gray scale
detections = face_detector_hog(image,1)
detections
for face in detections:
  cv2.rectangle(image, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 255), 1)

cv2_imshow(image)

# **CNN face detector**

In [ ]:
# Reinstall dlib to ensure compatibility with the current CUDA environment
!pip uninstall dlib -y
!pip install dlib-bin --upgrade

In [ ]:
# CNN face detector
import dlib
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
image = cv2.imread('/content/drive/MyDrive/img_cv/test_img/img/faces_2.jpg')
# Check if the image was loaded successfully
if image is None:
    raise FileNotFoundError("Error loading image. Check the file path.")

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_detector_cnn = dlib.cnn_face_detection_model_v1("/content/drive/MyDrive/img_cv/models/mmod_human_face_detector.dat")
# check the detector is exist
if face_detector_cnn is None:
  raise IOError("Unable to load the CNN face detection model")

# The parameters for the face_detector_cnn call are:
# image: This is the input image (in this case, the BGR color image loaded by cv2.imread).
#        The CNN face detector can work on both color and grayscale images, but it's common
#        to use the color image.
# 1: This is the 'upsampling' parameter. It indicates how many times to upsample the image
#    before running the detector. Upsampling the image can help detect smaller faces,
#    but it also increases processing time and memory usage. A value of 1 means the
#    image is upsampled once. A value of 0 means no upsampling.
detections = face_detector_cnn(image, 1)

for face in detections:
  l,t,r,b = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()
  cv2.rectangle(image, (l, t), (r, b), (0, 255, 255), 1)
  print(face.confidence)

cv2_imshow(image)

# **lbph face recognizing**

In [ ]:
# reinstall the package support for the lbph
!pip uninstall opencv-contrib-python -y
!pip install opencv-contrib-python

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/img_cv/dataset/archive/'

def get_img_data():
  paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)]
  ids = []
  faces = []
  for path in paths:
    if os.path.isfile(path) and path.lower().endswith('.gif'):
      img = Image.open(path).convert('L') # L is single scale color image
      img_np = np.array(img, 'uint8')
      id = int(os.path.split(path)[1].split('.')[0].replace('subject',''))
      ids.append(id)
      faces.append(img_np)

  return np.array(ids), faces

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# get the faces and id of the faces
ids, faces = get_img_data()


# create classifier instance and input training model to output
lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
lbph_classifier.train(faces, ids)
lbph_classifier.write('lbph_classifier.yml')

In [ ]:
# get the sample image
test_image = '/content/drive/MyDrive/img_cv/dataset/archive/subject10.sad.gif'

image = Image.open(test_image).convert('L')
image_np = np.array(image, 'uint8')
image_np

In [ ]:
# using the model to test the sample image
prediction = lbph_classifier.predict(image_np)
prediction
# param 1: id of the subject the model detect
# param 2: the confident point, the lower point, the better match
#result (3, 0,0)

# show the expected output (selected image id)
expected_output = int(os.path.split(test_image)[1].split('.')[0].replace('subject',''))
expected_output

In [ ]:
# test for all the sample image in folder
paths = [os.path.join('/content/drive/MyDrive/img_cv/dataset/archive', filename) for filename in os.listdir('/content/drive/MyDrive/img_cv/dataset/archive')]
predictions = []
expected_outputs = []

for path in paths:
  # Check if the file is a GIF before processing
  if os.path.isfile(path) and path.lower().endswith('.gif'):
    try:
      # image input for test model
      image = Image.open(path).convert('L')
      image_np = np.array(image, 'uint8')
      # using model the get the predicted sample id
      prediction, _ = lbph_classifier.predict(image_np)
      #
      expected_output = int(os.path.split(path)[1].split('.')[0].replace('subject',''))

      predictions.append(prediction)
      expected_outputs.append(expected_output)
    except Exception as e:
      print(f"Error processing file {path}: {e}")

# change result to array
predictions = np.array(predictions)
expected_outputs = np.array(expected_outputs)

In [ ]:
# check array of prediction and expected
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(expected_outputs, predictions)
accuracy

cm = confusion_matrix(expected_outputs, predictions)
cm

# draw the diagram for result
import seaborn

seaborn.heatmap(cm, annot=True)

# **Facial detection**

In [ ]:
# facial detection
import dlib
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image


# detector outline the face
face_detector = dlib.get_frontal_face_detector()
# detector for facial
point_detector = dlib.shape_predictor('/content/drive/MyDrive/img_cv/models/shape_predictor_68_face_landmarks.dat')
#get the test image
image =  cv2.imread('/content/drive/MyDrive/img_cv/test_img/img/faces_2.jpg')

#input the image to detector
face_detection = face_detector(image, 1)

for face in face_detection:
  # draw the rectangle for each face
  l,t,r,b = face.left(), face.top(), face.right(), face.bottom()
  cv2.rectangle(image, (l, t), (r, b), (0, 255, 255), 1)

  points = point_detector(image, face)
  for point in points.parts():
    cv2.circle(image, (point.x, point.y), 1, (0, 255, 0), 1)


cv2_imshow(image)

# **Detecting facial descriptor**

In [ ]:
# Reinstall dlib to ensure compatibility with the current CUDA environment
!pip uninstall dlib -y
!pip install dlib-bin --upgrade

In [2]:
# import the dependency
import os
import dlib
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from google.colab import drive
import numpy as np

In [3]:
# detector outline the face
face_detector = dlib.get_frontal_face_detector()
# detector for facial
point_detector = dlib.shape_predictor('/content/drive/MyDrive/img_cv/models/shape_predictor_68_face_landmarks.dat')
#face_description_extractor
face_description_extractor = dlib.face_recognition_model_v1('/content/drive/MyDrive/img_cv/models/dlib_face_recognition_resnet_model_v1.dat')

In [22]:
# execuate the descriptor for the sample data
index = {}
idx = 0
face_descriptors = None

paths = [os.path.join('/content/drive/MyDrive/img_cv/dataset/archive', filename) for filename in os.listdir('/content/drive/MyDrive/img_cv/dataset/archive')]
for path in paths:
  if os.path.isfile(path) and path.lower().endswith('.gif'):
    image = Image.open(path).convert('RGB')
    image_np = np.array(image, 'uint8')
    face_detection = face_detector(image_np, 1)
    for face in face_detection:
      l,t,r,b = face.left(), face.top(), face.right(), face.bottom()
      cv2.rectangle(image_np, (l, t), (r, b), (0, 255, 255), 1)

      points = point_detector(image_np, face)
      for point in points.parts():
        cv2.circle(image_np, (point.x, point.y), 1, (0, 255, 0), 1)

      face_descriptor = face_description_extractor.compute_face_descriptor(image_np, points)
      # print(len(face_descriptor))

      # init and add to the array of face_description
      if face_descriptors is None:
        face_descriptors = np.array(face_descriptor)
      else:
        face_descriptors = np.vstack((face_descriptors, np.array(face_descriptor)))

      # add index for each face
      index[idx] = {path}
      idx += 1

    # cv2_imshow(image_np)

In [ ]:
index

In [ ]:
# calculate the distance of the faces

# Get the path from the set stored in the index
image1_path = list(index[45])[0]
image2_path = list(index[55])[0]

image1 = Image.open(image1_path).convert('RGB')
image1_np = np.array(image1, 'uint8')
cv2_imshow(image1_np)

image2 = Image.open(image2_path).convert('RGB')
image2_np = np.array(image2, 'uint8')
cv2_imshow(image2_np)

np.linalg.norm(face_descriptors[45] - face_descriptors[55])

In [ ]:
distances = np.linalg.norm(face_descriptors[45] - face_descriptors, axis=1)
idx = 0
for distance in distances:
  if distance < 0.3:
    image = Image.open(list(index[idx])[0])
    image_np = np.array(image, 'uint8')
    cv2_imshow(image_np)
  idx +=1